In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten,BatchNormalization

from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
seed = 16
filename = '../fer2013.csv'
label_map = ['Enfado', 'Asco', 'Miedo', 'Alegria', 'Tristeza', 'Sorpresa', 'Neutral']
names=['emocion','pixels','uso']
df=pd.read_csv(filename,names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emocion,pixels,uso
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [3]:
def getData(filname):
    # Las imagenes 48x48 en grayscale (de 0 a 255)
    # N = 35887 (28709 de entrenamiento y 2304 de testeo)
    Y_train = []
    X_train = []
    Y_test = []
    X_test = []
    
    first = True
    uso = 'Training\n'
    for line in open(filname):
        #This condition skips the first condition
        if first:
            first = False
        else:
            row = line.split(',')
            condition = str(row[2])
            if condition == uso:
                Y_train.append(int(row[0]))
                X_train.append([int(p) for p in row[1].split()])
            else:
                Y_test.append(int(row[0]))
                X_test.append([int(p) for p in row[1].split()])
                
    X_train, Y_train = np.array(X_train)/255.0 , np.array(Y_train)
    X_test, Y_test = np.array(X_test)/255.0 , np.array(Y_test)

    return X_train, X_test, Y_train, Y_test 

In [4]:
X_train, X_test, Y_train, Y_test= getData(filename)

In [5]:
N_train = X_train.shape[0]
N_test = X_test.shape[0]
X_train = X_train.reshape(N_train, 48, 48,1)
X_test = X_test.reshape(N_test, 48, 48,1)

In [6]:
cont_train = [0,0,0,0,0,0,0]
for j in Y_train:
    cont_train[j] += 1
cont_test = [0,0,0,0,0,0,0]
for j in Y_test:
    cont_test[j] += 1

print(cont_train)
print(cont_test)

[3995, 436, 4097, 7215, 4830, 3171, 4965]
[958, 111, 1024, 1774, 1247, 831, 1233]


In [7]:
# one hot encode output
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
class_num = Y_test.shape[1]

In [8]:
model = Sequential()

model.add(Conv2D(6,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'tanh'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'tanh'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dense(class_num, activation='softmax',kernel_constraint=maxnorm(3)))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 44, 44, 6)         156       
_________________________________________________________________
average_pooling2d_1 (Average (None, 22, 22, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 16)        2416      
_________________________________________________________________
average_pooling2d_2 (Average (None, 9, 9, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1296)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               166016    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                

In [10]:
epochs = 10
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [11]:
np.random.seed(16)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Train on 28709 samples, validate on 7178 samples
Epoch 1/10
28709/28709 [==============================] - 50s 2ms/step - loss: 1.6697 - accuracy: 0.3410 - val_loss: 1.5881 - val_accuracy: 0.3833
Epoch 2/10
28709/28709 [==============================] - 43s 1ms/step - loss: 1.5376 - accuracy: 0.4046 - val_loss: 1.5153 - val_accuracy: 0.4189
Epoch 3/10
28709/28709 [==============================] - 41s 1ms/step - loss: 1.4635 - accuracy: 0.4391 - val_loss: 1.4880 - val_accuracy: 0.4308
Epoch 4/10
28709/28709 [==============================] - 40s 1ms/step - loss: 1.3973 - accuracy: 0.4681 - val_loss: 1.4770 - val_accuracy: 0.4391
Epoch 5/10
28709/28709 [==============================] - 40s 1ms/step - loss: 1.3345 - accuracy: 0.4936 - val_loss: 1.4843 - val_accuracy: 0.4315
Epoch 6/10
28709/28709 [==============================] - 41s 1ms/step - loss: 1.2741 - accuracy: 0.5191 - val_loss: 1.4963 - val_accuracy: 0.4450
Epoch 7/10
28709/2870

In [12]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 44.15%


In [13]:
model = Sequential()

model.add(Conv2D(6,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dense(class_num, activation='softmax',kernel_constraint=maxnorm(3)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 44, 44, 6)         156       
_________________________________________________________________
average_pooling2d_3 (Average (None, 22, 22, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 16)        2416      
_________________________________________________________________
average_pooling2d_4 (Average (None, 9, 9, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1296)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               166016    
_________________________________________________________________
dense_4 (Dense)              (None, 7)                

In [14]:
epochs = 10
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
np.random.seed(16)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Train on 28709 samples, validate on 7178 samples
Epoch 1/10
28709/28709 [==============================] - 42s 1ms/step - loss: 1.6988 - accuracy: 0.3255 - val_loss: 1.6037 - val_accuracy: 0.3911
Epoch 2/10
28709/28709 [==============================] - 42s 1ms/step - loss: 1.5601 - accuracy: 0.3975 - val_loss: 1.5215 - val_accuracy: 0.4185
Epoch 3/10
28709/28709 [==============================] - 43s 1ms/step - loss: 1.4797 - accuracy: 0.4263 - val_loss: 1.4614 - val_accuracy: 0.4374
Epoch 4/10
28709/28709 [==============================] - 42s 1ms/step - loss: 1.4183 - accuracy: 0.4529 - val_loss: 1.4156 - val_accuracy: 0.4600
Epoch 5/10
28709/28709 [==============================] - 43s 2ms/step - loss: 1.3608 - accuracy: 0.4799 - val_loss: 1.3876 - val_accuracy: 0.4681
Epoch 6/10
28709/28709 [==============================] - 43s 1ms/step - loss: 1.3128 - accuracy: 0.5004 - val_loss: 1.3590 - val_accuracy: 0.4738
Epoch 7/10
28709/28709 [==============================] - 43s 2ms/ste

In [15]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 50.49%


In [16]:
model = Sequential()

model.add(Conv2D(6,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16,(5,5),input_shape=X_train.shape[1:],padding = 'valid', strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dense(class_num, activation='softmax',kernel_constraint=maxnorm(3)))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 44, 44, 6)         156       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 6)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 18, 16)        2416      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1296)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               166016    
_________________________________________________________________
dense_6 (Dense)              (None, 7)                

In [17]:
epochs = 10
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
np.random.seed(16)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64)

Train on 28709 samples, validate on 7178 samples
Epoch 1/10
28709/28709 [==============================] - 48s 2ms/step - loss: 1.6717 - accuracy: 0.3393 - val_loss: 1.5519 - val_accuracy: 0.4083
Epoch 2/10
28709/28709 [==============================] - 47s 2ms/step - loss: 1.4928 - accuracy: 0.4276 - val_loss: 1.4569 - val_accuracy: 0.4418
Epoch 3/10
28709/28709 [==============================] - 49s 2ms/step - loss: 1.3988 - accuracy: 0.4655 - val_loss: 1.3806 - val_accuracy: 0.4677
Epoch 4/10
28709/28709 [==============================] - 45s 2ms/step - loss: 1.3351 - accuracy: 0.4915 - val_loss: 1.3530 - val_accuracy: 0.4777
Epoch 5/10
28709/28709 [==============================] - 47s 2ms/step - loss: 1.2806 - accuracy: 0.5150 - val_loss: 1.3269 - val_accuracy: 0.4936
Epoch 6/10
28709/28709 [==============================] - 46s 2ms/step - loss: 1.2277 - accuracy: 0.5344 - val_loss: 1.3111 - val_accuracy: 0.4965
Epoch 7/10
28709/28709 [==============================] - 46s 2ms/ste

In [18]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 51.30%
